In [1]:
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import tensorflow as tf
import cv2
import os
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM,  Conv2DTranspose, Input, Conv2D, MaxPooling2D, UpSampling2D, \
TimeDistributed, ConvLSTM2D, Reshape, Lambda, Flatten, Concatenate, Conv3D, MaxPooling3D, UpSampling3D, Conv3DTranspose, \
Dropout, concatenate
from tensorflow.keras.backend import clear_session

In [2]:
data_dir= "M:\\Exposure_dataset\\INPUT"
GT_dir= "M:\\Exposure_dataset\\GT"

In [3]:
cnt=1
cur_dir=""
cnt2=0

In [4]:
# for img in os.listdir(data_dir):
#     if (cnt%5==1):
#         cnt2+=1
#         os.mkdir(os.path.join(data_dir, f'img_{cnt2}'))
#         cur_dir= os.path.join(data_dir, f'img_{cnt2}')
#     shutil.move(os.path.join(data_dir, img), cur_dir)        
#     cnt+=1

In [5]:
# cnt=1
# for img in os.listdir(GT_dir):
#     os.mkdir(os.path.join(GT_dir, f'img_{cnt}'))
#     cur_dir= os.path.join(GT_dir, f'img_{cnt}')
#     shutil.move(os.path.join(GT_dir, img), cur_dir)        
#     cnt+=1

In [6]:
img_width, img_height = 256, 256
batch_size = 1
distorted_datagen = ImageDataGenerator(rescale=1.0 / 255)
enhanced_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [7]:
distorted_generator = distorted_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None ,  # This means the generator will only yield images, not labels
    shuffle=False  # You can set this to False if you want to preserve the order
)

Found 0 images belonging to 0 classes.


In [8]:
enhanced_generator = enhanced_datagen.flow_from_directory(
    GT_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 0 images belonging to 0 classes.


In [10]:
def combined_generator(X_generator, y_generator):
    while True:
        X_batch = next(X_generator)
        y_batch = next(y_generator)
        yield X_batch, y_batch
        

In [11]:
# Example usage
combined_gen = combined_generator(distorted_generator, enhanced_generator)

In [12]:
input_shape = (128, 128, 3) 

In [13]:
inputs = Input(shape=input_shape)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

# Bottom
conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
drop5 = Dropout(0.5)(conv5)

# Decoder
up6 = UpSampling2D(size=(2, 2))(drop5)
up6 = Conv2D(512, 2, activation='relu', padding='same')(up6)
merge6 = concatenate([drop4, up6], axis=3)
conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
up7 = UpSampling2D(size=(2, 2))(conv6)

In [14]:

up7 = Conv2D(256, 2, activation='relu', padding='same')(up7)
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

up8 = UpSampling2D(size=(2, 2))(conv7)
up8 = Conv2D(128, 2, activation='relu', padding='same')(up8)
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

up9 = UpSampling2D(size=(2, 2))(conv8)
up9 = Conv2D(64, 2, activation='relu', padding='same')(up9)
merge9 = concatenate([conv1, up9], axis=3)
conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

outputs = Conv2D(3, 1, activation='linear')(conv9)
model = Model(inputs=inputs, outputs=outputs)

In [15]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

In [18]:
history = model.fit(
    x= distorted_generator,
    y= enhanced_generator,
    epochs=10,
    steps_per_epoch=len(distorted_generator),
)

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.